In [171]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import scipy.integrate as integrate

import warnings
warnings.filterwarnings(action='ignore')

import time
import math
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import os

c = 299792.458
sign = 8

In [50]:
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(executable_path="../chromedriver",
                              options=chrome_options)
    return driver

In [190]:
def make_query(ra, dec, r200, dist, z_upper, z_lower):
    theta = np.arcsin(r200 / dist) * 60 * 180 / np.pi
    query = f"""SELECT
   p.objid, p.ra, p.dec, p.l, p.b,
   p.petroMag_u-p.extinction_u AS mag_u,
   p.petroMag_g-p.extinction_g AS mag_g,
   p.petroMag_r-p.extinction_r AS mag_r,
   p.petroMag_i-p.extinction_i AS mag_i,
   p.petroMag_z-p.extinction_z AS mag_z,
   s.z as redshift, s.zErr as redshift_err,
   s.velDisp, s.velDispErr,
   passive.logmass as passive_logmass,
   ROUND(pca.mstellar_median, 2) as pca_logmass,
   ROUND(passive.logmass-passive.minlogmass, 3) as logmass_lowerErr,
   ROUND(passive.maxlogmass-passive.logmass, 3) as logmass_upperErr,
   passive.absMagK,
   pca.vdisp_median as pca_vdisp, pca.vdisp_err as pca_vdisp_err,
   zoo.p_el_debiased as frac_ellip,
   zoo.p_cs_debiased as frac_spiral
FROM Galaxy AS p
   JOIN dbo.fGetNearbyObjEq({ra}, {dec}, {theta}) AS GN ON GN.objid = p.objid
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   LEFT JOIN stellarMassPassivePort AS passive ON passive.specobjid = s.specobjid
   LEFT JOIN stellarMassPCAWiscM11 AS pca ON pca.specobjid = s.specobjid
   LEFT JOIN zooSpec AS zoo ON zoo.specobjid = s.specobjid
WHERE
   s.class = 'GALAXY'
   AND s.z BETWEEN {z_lower} AND {z_upper}"""
    return query

In [84]:
query = make_query(1,1,0.5,1)
print(query)

SELECT
   p.objid, p.ra, p.dec, p.l, p.b,
   p.petroMag_u-p.extinction_u AS mag_u,
   p.petroMag_g-p.extinction_g AS mag_g,
   p.petroMag_r-p.extinction_r AS mag_r,
   p.petroMag_i-p.extinction_i AS mag_i,
   p.petroMag_z-p.extinction_z AS mag_z,
   s.z as redshift,
   z.p_el as elliptical, 
   z.p_cw as spiralclock, z.p_acw as spiralanticlock,
   z.p_edge as edgeon, z.p_mg as merger
FROM Galaxy AS p
   JOIN SpecObj AS s ON s.bestobjid = p.objid
   JOIN ZooNoSpec AS z ON z.objid = p.objid
WHERE
   s.class = 'GALAXY'
   AND p.ra BETWEEN -28.999999999999996 AND 30.999999999999996
   AND p.dec BETWEEN -28.999999999999996 AND 30.999999999999996


In [130]:
def get_csv(driver, query):
    driver.find_element(By.XPATH, '//*[@id="clear"]').click()
    driver.find_element(By.XPATH, '//*[@id="cmd"]').send_keys(query)
    driver.find_element(By.XPATH, '//*[@id="transp"]/form/table/tbody/tr[3]/td/table/tbody/tr/td[2]/table/tbody/tr[1]/td[4]/input').click()
    driver.find_element(By.XPATH, '/html/body/div[4]/form/table/tbody/tr[3]/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/input').click()
    
    time.sleep(np.random.randint(15,20))
    
def get_html(driver, query):
    driver.find_element(By.XPATH, '//*[@id="clear"]').click()
    driver.find_element(By.XPATH, '//*[@id="cmd"]').send_keys(query)
    driver.find_element(By.XPATH, '/html/body/div[4]/form/table/tbody/tr[3]/td/table/tbody/tr/td[1]/table/tbody/tr[6]/td/input').click()
    
    time.sleep(np.random.randint(15,20))

In [180]:
def digitScaler(num, sign):
    digit = math.floor(np.log10(num))
    n = num / 10**digit
    n = round(n, sign-1)
    return n * math.pow(10,digit)

def redshift(v_r):
    # calculating relativistic redshift with radial velocity
    return np.sqrt((c + v_r)/(c - v_r)) - 1

def radial_v(z):
    # calculating relativistic radial velocity with redshift
    return c * ((z+1)**2 - 1)/((z+1)**2 + 1)

def dimless_h(z, h_0=1):
    # Calculating dimesionless hubble constant, h(z)
    # According to the GalWCat19 paper,
    # Omega(r, m, k, gamma) = (0, 0.3, 0, 0.7)
    # h_0 = 0.7
    return h_0 * np.sqrt(0.3*(1+z)**3 + 0.7)

def comv_dist(z):
    return (c / 100) * integrate.quad(lambda x: 1/dimless_h(x), 0, z)[0]

def angular_dist(delta_ra, delta_dec):
    tr = np.pi / 180
    return np.arccos(np.cos(delta_ra * tr) * np.cos(delta_dec * tr)) / tr

In [165]:
header = ['ra', 'dec', 'z', 'v_r',
          'comv_dist', 'r_200', 'N_200', 'sigma_200',
          'sigma_200_lowerErr', 'sigma_200_upperErr', 'M_200', 'M_200_Err',
          'R_s', 'R_s_Err', 'M_s', 'M_s_Err']
cls_info = pd.read_csv('./GalWCls19_summary.csv',index_col=0)
cls_info.columns = header

In [178]:
cls1 = cls_info.iloc[0,:]

v = cls1['v_r']
z_calc = redshift(v)
v_calc = radial_v(cls1['z'])
print(digitScaler(v_calc, sign))
print(v)
print(cls1['comv_dist'])
print(digitScaler(comv_dist(cls1['z']), sign))

21137.193
21137.193
215.71431
215.71430999999998


In [187]:
print(digitScaler(angular_dist(1, 1), sign_fig))

1.4141777
1.4142135623730951


In [191]:
url = "http://skyserver.sdss.org/dr13/en/tools/search/sql.aspx"
driver = set_chrome_driver()
driver.get(url)

for idx, cls in cls_info.iterrows():
    ra = cls['ra']
    dec = cls['dec']
    dist = cls['comv_dist']
    r200 = cls['r_200']
    v_r = cls['v_r']
    sigma_upper = cls['sigma_200'] + cls['sigma_200_upperErr']
    
    z_upper = redshift(v_r + sigma_upper)
    z_lower = redshift(v_r - sigma_upper)
    
    query = make_query(ra, dec, r200, dist, z_upper, z_lower)
#     get_html(driver, query)
    get_csv(driver, query)
    
#     if idx > 0:
#         break